In [1]:
#Import
from flask import Flask, request, render_template, redirect, url_for, flash, Blueprint
from flask_bootstrap import Bootstrap
app = Flask(__name__)
Bootstrap(app)
import flask_ipywidgets
import pymongo
from pymongo import MongoClient
from ipywidgets.embed import embed_minimal_html

from flask_wtf import FlaskForm
from flask_wtf.csrf import CSRFProtect
from wtforms import StringField, PasswordField, BooleanField, SubmitField
from wtforms.validators import DataRequired

from flask import g
from flask.sessions import SecureCookieSessionInterface
from flask_login import user_loaded_from_request
from flask_login import LoginManager
login_manager = LoginManager()


In [2]:
#Create client
client = MongoClient("localhost",27017)

In [3]:
#Create db
db = client["testDB"]

In [4]:
posts = db.posts

In [5]:
#Reset
x = posts.delete_many({})

In [6]:
users = [{"user": "Admin",
        "password": "topSecret"},
         {"user": "Uschi",
         "password": "secret01"}]

result = posts.insert_many(users)

In [7]:
#Logged in User
class Users:
    def __init__(self, name, passwort):
        self.name = name
        self.passwort = passwort

In [8]:
#Config and Start App
app = Flask(__name__)
login_manager.init_app(app)
csrf = CSRFProtect(app)
app.config['WTF_CSRF_CHECK_DEFAULT'] = False
app.config['WTF_CSRF_ENABLED'] = False
app.config['SECRET_KEY'] = 'you-will-never-guess'

In [9]:
login_manager.init_app(app)
login_manager.session_protection = None
#REMEMBER_COOKIE_DURATION = timedelta(minutes=0)

@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)

@app.route('/', methods=['GET', 'POST'])
def login():
    # Here we use a class of some kind to represent and validate our
    # client-side form data. For example, WTForms is a library that will
    # handle this for us, and we use a custom LoginForm to validate.
    form = LoginForm()
    if form.validate_on_submit():
        flash('Login requested for user {}'.format(
            form.username.data))
        u = Users(form.username.data, form.password.data)
        for post in posts.find({'user': form.username.data, 'password': form.password.data}):
            print(post)
            return redirect(url_for('index', username=form.username.data))
    return render_template('login.html', title='Sign In', form=form)

In [10]:
#Form
class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Sign In')

In [11]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    username = request.args['username']
    user = {'username': username}
    return render_template('index.html', title='Sign In', user=user)

In [ ]:
#Run Server
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
